In [ ]:
#Individual Data extraction and processing

# Essential imports for multivariate modeling and Binance data handling
import os
import glob
import logging
import random
import threading
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

from binance.client import Client
from binance.exceptions import BinanceAPIException, BinanceRequestException

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import GRU, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# Set your Binance API key and secret (leave blank for public endpoints)
api_key = ''
api_secret = ''

# Create the client and verify the connection
try:
    client = Client(api_key, api_secret)
    # Test connectivity (ping)
    status = client.ping()
    if status == {}:
        print("Connection to Binance API successful!")
    else:
        print("Unexpected ping response:", status)
except (BinanceAPIException, BinanceRequestException) as e:
    print(f"Binance API connection failed: {e}")
except Exception as e:
    print(f"Connection failed: {e}")

# data timeframe
start_str = "2023-12-01 00:00:00"
end_str = "2024-12-31 23:59:59"
start_dt = datetime.datetime.strptime(start_str, "%Y-%m-%d %H:%M:%S")
end_dt = datetime.datetime.strptime(end_str, "%Y-%m-%d %H:%M:%S")

# Pull all data in one go
response = client.get_historical_klines(
    "BTCUSDT",
    Client.KLINE_INTERVAL_1MINUTE,
    start_str,
    end_str
)

# Convert to DataFrame
def process_klines(response):
    """Convert raw kline data into a pandas DataFrame."""
    columns = [
        "open_time", "open", "high", "low", "close", "volume",
        "close_time", "quote_asset_volume", "number_of_trades",
        "taker_buy_base_asset_volume", "taker_buy_quote_asset_volume", "ignore"
    ]
    df = pd.DataFrame(response, columns=columns)
    df["open_time"] = pd.to_datetime(df["open_time"], unit='ms')
    df.set_index("open_time", inplace=True)
    # Convert numeric columns
    for col in ["open", "high", "low", "close", "volume"]:
        df[col] = df[col].astype(float)
    df.sort_index(ascending=True, inplace=True)
    return df

def compute_rsi(series, period=14*24):
    """Compute the Relative Strength Index (RSI)."""
    delta = series.diff()
    gain = delta.clip(lower=0).ewm(alpha=1/period, adjust=False).mean()
    loss = -delta.clip(upper=0).ewm(alpha=1/period, adjust=False).mean()
    rs = gain / (loss + 1e-10)
    return 100 - (100 / (1 + rs))

def compute_roc(series, period=24):
    """Compute the Rate of Change (ROC)."""
    return series.pct_change(periods=period) * 100

def compute_ema(series, period=12*24):
    """Compute the Exponential Moving Average (EMA)."""
    return series.ewm(span=period, adjust=False).mean()

# Process the raw data
df = process_klines(response)

# Calculate indicators
df['RSI_14'] = compute_rsi(df['close'], period=14*24)
df['ROC_1'] = compute_roc(df['close'], period=24)
df['EMA_12'] = compute_ema(df['close'], period=12*24)

# Select only the desired columns
output_df = df[['close', 'volume', 'RSI_14', 'ROC_1', 'EMA_12']]

# Save to CSV
output_df.to_csv("sol_usdt_hourly_indicators.csv")

# Display the first few rows to the user
import ace_tools as tools; tools.display_dataframe_to_user(name="SOL/USDT Price & Indicators", dataframe=output_df.head(10))


In [ ]:
# PCA based indexing
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn import preprocessing
import matplotlib.pyplot as plt
import os

# List of DAO coin symbols
dao_coins = ['btc', 'eth', 'sol', 'avax', 'bnb']  # Example DAO coins

# Create data directory path
data_dir = 'data'

# Load and preprocess data
dfs = []
for symbol in dao_coins:
    # Load each coin's data from the data directory
    filename = os.path.join(data_dir, f"{symbol}usdt_hourly_indicators.csv")
    df = pd.read_csv(filename)
    
    # Convert open_time to datetime and set as index
    df['open_time'] = pd.to_datetime(df['open_time'])
    df.set_index('open_time', inplace=True)
    
    # Select only close price and volume
    df = df[['close', 'volume']]
    df.columns = [f'{symbol}_close', f'{symbol}_volume']
    dfs.append(df)

# Combine all data into one DataFrame
combined_df = pd.concat(dfs, axis=1)

# Forward fill missing values (if any)
combined_df.ffill(inplace=True)

# Drop any remaining NA values
combined_df.dropna(inplace=True)

# Extract just close prices and volumes for PCA analysis
close_prices = combined_df[[col for col in combined_df.columns if 'close' in col]]
volume_data = combined_df[[col for col in combined_df.columns if 'volume' in col]]

# Standardize the data
X_close = preprocessing.StandardScaler().fit_transform(close_prices)
X_volume = preprocessing.StandardScaler().fit_transform(volume_data)

# Combine close prices and volumes (50/50 weighting)
X_combined = np.hstack([X_close, X_volume])

# Perform PCA
pca = PCA(n_components=3)
pca_results = pca.fit_transform(X_combined)

# Get sum of weights for first 3 principal components
n_coins = len(dao_coins)
close_weights = pca.components_[:, :n_coins]
volume_weights = pca.components_[:, n_coins:]

# Combine weights (equal weighting for price and volume)
combined_weights = np.sum(close_weights + volume_weights, axis=0)

# Create index fund weights (normalized)
index_fund_weights = combined_weights / np.sum(np.abs(combined_weights))
index_fund_tickers = dao_coins

# New version (raw):
raw_index_close = (close_prices * index_fund_weights).sum(axis=1)
raw_index_volume = (volume_data * index_fund_weights).sum(axis=1)

# Save both raw and normalized versions
index_df = pd.DataFrame({
    'close_raw': raw_index_close,
    'volume_raw': raw_index_volume,
    'close_norm': raw_index_close / raw_index_close.iloc[0],  # Normalized
    'volume_norm': raw_index_volume / raw_index_volume.mean()  # Normalized
})

# Save to CSV
output_filename = 'lyrusdt_hourly_Indricators.csv'
index_df.to_csv(output_filename)
print(f"Index data saved to {output_filename}")

# Visualization 1: Index Fund Weights
plt.figure(figsize=(12, 6))
plt.bar(index_fund_tickers, index_fund_weights, color='skyblue', edgecolor='black')
plt.title('DAO Index Fund Weights', fontsize=16)
plt.xlabel('DAO Coin', fontsize=12)
plt.ylabel('Weight in Index', fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.savefig('DAO_Index_Weights.png')
plt.show()

# Create figure with 3 subplots
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(14, 14), sharex=True)

# 1. Plot RAW PRICE comparison
for coin in dao_coins:
    ax1.plot(close_prices[f'{coin}_close'], 
             label=f'{coin}', alpha=0.5, linewidth=1)
ax1.plot(raw_index_close, 
         label='DAO Index (Raw)', color='black', linewidth=2.5)
ax1.set_title('Raw Price Comparison', fontsize=14)
ax1.set_ylabel('Absolute Price', fontsize=12)
ax1.legend(loc='upper left', fontsize=10)
ax1.grid(linestyle='--', alpha=0.5)
ax1.ticklabel_format(style='plain', axis='y')  # Disable scientific notation

# 2. Plot NORMALIZED comparison
for coin in dao_coins:
    ax2.plot(close_prices[f'{coin}_close']/close_prices[f'{coin}_close'].iloc[0],
             label=f'{coin}', alpha=0.5, linewidth=1)
ax2.plot(raw_index_close/raw_index_close.iloc[0],
         label='DAO Index (Normalized)', color='blue', linewidth=2.5, linestyle='--')
ax2.set_title('Normalized Price Comparison (Base=100)', fontsize=14)
ax2.set_ylabel('Normalized Price', fontsize=12)
ax2.legend(loc='upper left', fontsize=10)
ax2.grid(linestyle='--', alpha=0.5)

# 3. Plot Volume comparison (normalized)
for coin in dao_coins:
    ax3.plot(volume_data[f'{coin}_volume']/volume_data[f'{coin}_volume'].mean(),
             label=f'{coin}', alpha=0.3, linewidth=0.8)
ax3.plot(raw_index_volume/raw_index_volume.mean(),
         label='DAO Index Volume', color='red', linewidth=2)
ax3.set_title('Normalized Volume Comparison', fontsize=14)
ax3.set_xlabel('Date', fontsize=12)
ax3.set_ylabel('Volume (Mean=1.0)', fontsize=12)
ax3.legend(loc='upper left', fontsize=10)
ax3.grid(linestyle='--', alpha=0.5)

plt.tight_layout()
plt.savefig('DAO_Index_Comparison.png', dpi=300, bbox_inches='tight')
plt.show()

# Correlation plot with ACTUAL values
plt.figure(figsize=(12, 6))
plt.scatter(raw_index_close, raw_index_volume, 
            alpha=0.6, c='green', s=20, edgecolor='k')
plt.title('Raw Index Price vs Volume Correlation', fontsize=16)
plt.xlabel('Absolute Close Price', fontsize=12)
plt.ylabel('Absolute Volume', fontsize=12)
plt.grid(linestyle='--', alpha=0.7)
plt.tight_layout()
plt.savefig('DAO_Index_Correlation_Raw.png', dpi=300)
plt.show()

# Bonus: Correlation plot with NORMALIZED values
plt.figure(figsize=(12, 6))
plt.scatter(raw_index_close/raw_index_close.iloc[0], 
            raw_index_volume/raw_index_volume.mean(),
            alpha=0.6, c='purple', s=20, edgecolor='k')
plt.title('Normalized Index Price vs Volume Correlation', fontsize=16)
plt.xlabel('Normalized Close Price', fontsize=12)
plt.ylabel('Normalized Volume', fontsize=12)
plt.grid(linestyle='--', alpha=0.7)
plt.tight_layout()
plt.savefig('DAO_Index_Correlation_Norm.png', dpi=300)
plt.show()


In [ ]:
# Model Training -Model Evaluation - Data extraction - Data Processing

# Essential imports for multivariate modeling and Binance data handling
import os
import glob
import logging
import random
import threading
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

from binance.client import Client
from binance.exceptions import BinanceAPIException, BinanceRequestException

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import GRU, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# Set your Binance API key and secret (leave blank for public endpoints)
api_key = ''
api_secret = ''

# Create the client and verify the connection
try:
    client = Client(api_key, api_secret)
    # Test connectivity (ping)
    status = client.ping()
    if status == {}:
        print("Connection to Binance API successful!")
    else:
        print("Unexpected ping response:", status)
except (BinanceAPIException, BinanceRequestException) as e:
    print(f"Binance API connection failed: {e}")
except Exception as e:
    print(f"Connection failed: {e}")

# --- Configuration ---
CONFIG = {
    'WINDOW_SIZE': 12,
    'BATCH_SIZE': 32,
    'EPOCHS': 50,
    'LEARNING_RATE': 0.001,
    'VAL_SPLIT': 0.1,
    'TRAIN_SPLIT': 0.8,
    'PATIENCE': 15,
    'MAX_DATE': "2023-12-31 23:59:59",
    'DATA_FILE_PATTERN': "*_hourly_indicators.csv",
    'MODEL_FILE_PATTERN': "model_{symbol}.keras",
    'LOG_LEVEL': logging.INFO
}

logging.basicConfig(level=CONFIG['LOG_LEVEL'], format='[%(levelname)s] %(message)s')

SEED = 42

def set_seed(seed=SEED):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

set_seed(SEED)

# --- Utility Functions ---
def list_csv_files(pattern=None):
    directory = "Data"
    pattern = pattern or CONFIG['DATA_FILE_PATTERN']
    return glob.glob(os.path.join(directory, pattern))

def select_from_list(options, prompt_msg):
    print("\n" + prompt_msg)
    for i, option in enumerate(options):
        print(f"{i}. {option}")
    
    while True:
        user_input = input(f"Enter your choice (0-{len(options)-1}): ").strip()
        try:
            choice = int(user_input)
            if 0 <= choice < len(options):
                return options[choice]
            print(f"Please enter a number between 0 and {len(options)-1}")
        except ValueError:
            print("Please enter a valid number.")

def text_input_widget(prompt):
    return input(prompt + ": ").strip()

def yes_no_widget(prompt):
    while True:
        user_input = input(prompt + " (y/n): ").strip().lower()
        if user_input in ('y', 'n'):
            return user_input
        print("Please enter 'y' or 'n'.")

def save_plot(fig, filename, symbol=None):
    if symbol:
        base, ext = os.path.splitext(filename)
        filename = f"{symbol}_{base}{ext}"
    fig.savefig(filename)
    logging.info(f"Plot saved to {filename}")

def save_metrics(metrics_df, filename, symbol=None):
    if symbol:
        base, ext = os.path.splitext(filename)
        filename = f"{symbol}_{base}{ext}"
    metrics_df.to_csv(filename, index=False)
    logging.info(f"Metrics saved to {filename}")

# --- Data Loading and Feature Engineering ---
def process_klines(response):
    columns = [
        "open_time", "open", "high", "low", "close", "volume",
        "close_time", "quote_asset_volume", "number_of_trades",
        "taker_buy_base_asset_volume", "taker_buy_quote_asset_volume", "ignore"
    ]
    df = pd.DataFrame(response, columns=columns)
    df["open_time"] = pd.to_datetime(df["open_time"], unit='ms')
    df.set_index("open_time", inplace=True)
    df = df[["close", "volume"]].astype(float)
    df = df.sort_index(ascending=True)
    return df

def compute_rsi(series, period=14*24):
    delta = series.diff()
    gain = delta.clip(lower=0).ewm(alpha=1/period, adjust=False).mean()
    loss = -delta.clip(upper=0).ewm(alpha=1/period, adjust=False).mean()
    rs = gain / (loss + 1e-10)
    return 100 - (100 / (1 + rs))

def compute_roc(series, period=24):
    return series.pct_change(periods=period) * 100

def compute_ema(series, period=12*24):
    return series.ewm(span=period, adjust=False).mean()

def add_indicators(df):
    df['RSI_14'] = compute_rsi(df['close'], period=14*24)
    df['ROC_1'] = compute_roc(df['close'], period=24)
    df['EMA_12'] = compute_ema(df['close'], period=12*24)
    return df

def load_or_download_data(client=None):
    csv_files = list_csv_files()
    options = csv_files + ["Pull new data"]
    selected_option = select_from_list(options, "Select data file or pull new data:")
    
    if selected_option != "Pull new data":
        logging.info(f"Loading data from: {selected_option}")
        # Extract first 3 characters from filename (without extension)
        filename_base = os.path.splitext(os.path.basename(selected_option))[0]
        symbol_or_prefix = filename_base[:6]  # First 3 characters
        
        # Load only the basic columns (close and volume)
        df = pd.read_csv(selected_option, parse_dates=["open_time"], index_col="open_time")
        
        # Ensure we have the required basic columns
        if 'close' not in df.columns or 'volume' not in df.columns:
            raise ValueError("Loaded CSV must contain at least 'close' and 'volume' columns")
            
        # Keep only close and volume (in case the file has other columns)
        df = df[['close', 'volume']].astype(float)
        
        # Calculate indicators
        df = add_indicators(df)
        
        # Save back to the same file (overwrite)
        df.to_csv(selected_option)
        logging.info(f"Indicators calculated and data saved back to: {selected_option}")
        return df, symbol_or_prefix
    
    # Pull new data
    symbol = text_input_widget("Enter the symbol of the crypto pair (e.g., BTCUSDT)").upper()
    if client is None or not symbol:
        raise ValueError("Binance client and symbol must be provided for download.")
    
    try:
        client.get_symbol_info(symbol)
    except Exception:
        raise ValueError(f"Symbol '{symbol}' isn't recognized or not available on Binance.")
    
    start_str = "2017-01-01 00:00:00"
    end_str = "2025-01-01 00:00:00"
    logging.info("Downloading data (this may take a while)...")
    response = client.get_historical_klines(
        symbol,
        client.KLINE_INTERVAL_1HOUR,
        start_str,
        end_str
    )
    logging.info("Data pulled.")
    df = process_klines(response)
    df = add_indicators(df)
    filename = f"{symbol.lower()}_hourly_indicators.csv"
    df.to_csv(filename)
    logging.info(f"Data saved to {filename}")
    return df, symbol

# --- Scaling Utilities ---
class FeatureScaler:
    def __init__(self, minmax_cols, std_cols):
        self.minmax_cols = minmax_cols
        self.std_cols = std_cols
        self.mm_scaler = MinMaxScaler()
        self.std_scaler = StandardScaler()
    
    def fit(self, df):
        self.mm_scaler.fit(df[self.minmax_cols])
        self.std_scaler.fit(df[self.std_cols])
    
    def transform(self, df):
        scaled = df.copy()
        scaled[self.minmax_cols] = self.mm_scaler.transform(df[self.minmax_cols])
        scaled[self.std_cols] = self.std_scaler.transform(df[self.std_cols])
        return scaled
    
    def inverse_transform_close(self, arr):
        zeros = np.zeros((arr.shape[0], len(self.minmax_cols)-1))
        arr_full = np.concatenate([arr.reshape(-1, 1), zeros], axis=1)
        return self.mm_scaler.inverse_transform(arr_full)[:, 0]

# --- Data Preparation ---
def split_and_scale(features, scaler, config):
    max_date = pd.Timestamp(config['MAX_DATE'])
    features = features.loc[:max_date]
    n_cut = len(features) - config['WINDOW_SIZE'] + 1
    train_end = int(n_cut * config['TRAIN_SPLIT'])
    val_end = train_end + int(n_cut * config['VAL_SPLIT'])
    scaler.fit(features.iloc[:train_end])
    train_data = scaler.transform(features.iloc[:train_end])
    val_data = scaler.transform(features.iloc[train_end:val_end])
    test_data = scaler.transform(features.iloc[val_end:n_cut])
    return train_data, val_data, test_data, train_end, val_end, n_cut

def df_to_x_y_multivariate(features, window_size):
    features_numpy = features.to_numpy()
    x, y = [], []
    for i in range(len(features) - window_size):
        x.append(features_numpy[i:i + window_size])
        y.append(features_numpy[i + window_size][0])
    return np.array(x), np.array(y)

# --- Model Definition ---
def build_gru_model(input_shape, lr):
    model = Sequential([
        GRU(64, return_sequences=True, input_shape=input_shape),
        GRU(32),
        Dense(16, activation='relu'),
        Dense(1, activation='linear')
    ])
    model.compile(
        loss=MeanSquaredError(),
        optimizer=Adam(learning_rate=lr),
        metrics=[RootMeanSquaredError()]
    )
    return model

# --- Metrics and Plotting ---
def compute_metrics(results, name):
    rmse = np.sqrt(mean_squared_error(results['actuals'], results['predictions']))
    mae = mean_absolute_error(results['actuals'], results['predictions'])
    rmse_inv = np.sqrt(mean_squared_error(results['actuals_inv'], results['predictions_inv']))
    mae_inv = mean_absolute_error(results['actuals_inv'], results['predictions_inv'])
    mape_inv = mean_absolute_percentage_error(results['actuals_inv'], results['predictions_inv']) * 100
    return [
        [f'{name} (scaled)', rmse, mae, "N/A"],
        [f'{name} (inversed)', rmse_inv, mae_inv, mape_inv]
    ]

def plot_results(ax, idx, preds, actuals, title, ylabel=None):
    ax.plot(idx[:1000], preds[:1000], label='Predictions', color='orange')
    ax.plot(idx[:1000], actuals[:1000], label='Actuals', color='blue')
    ax.set_title(title)
    if ylabel:
        ax.set_ylabel(ylabel)
    ax.legend()

def plot_all_results(train_results, val_results, test_results, config):
    fig, axes = plt.subplots(3, 2, figsize=(16, 12))
    plot_results(axes[0, 0], range(1000), train_results['predictions'], train_results['actuals'], 'Train Predictions vs Actuals (Scaled)')
    plot_results(axes[0, 1], train_results['index'], train_results['predictions_inv'], train_results['actuals_inv'], 'Train Predictions vs Actuals (Inversed)', ylabel='Close Price')
    plot_results(axes[1, 0], range(1000), val_results['predictions'], val_results['actuals'], 'Validation Predictions vs Actuals (Scaled)')
    plot_results(axes[1, 1], val_results['index'], val_results['predictions_inv'], val_results['actuals_inv'], 'Validation Predictions vs Actuals (Inversed)', ylabel='Close Price')
    plot_results(axes[2, 0], range(1000), test_results['predictions'], test_results['actuals'], 'Test Predictions vs Actuals (Scaled)')
    plot_results(axes[2, 1], test_results['index'], test_results['predictions_inv'], test_results['actuals_inv'], 'Test Predictions vs Actuals (Inversed)', ylabel='Close Price')
    plt.tight_layout()
    save_plot(fig, "predictions_vs_actuals.png")
    plt.show()

def plot_training_loss(history):
    fig_loss = plt.figure(figsize=(8, 4))
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Val Loss')
    plt.title('GRU Model Loss Curves')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.tight_layout()
    save_plot(fig_loss, "training_loss_curve.png")
    plt.show()

# --- Evaluation Logic ---
def get_results(model, x, y, scaler, data_idx):
    preds = model.predict(x, verbose=0).flatten()
    preds_inv = scaler.inverse_transform_close(preds)
    actuals_inv = scaler.inverse_transform_close(y)
    idx = data_idx.values if hasattr(data_idx, 'values') else data_idx
    return {
        'predictions': preds,
        'actuals': y,
        'predictions_inv': preds_inv,
        'actuals_inv': actuals_inv,
        'index': idx
    }

def evaluate_and_plot(model, x_train, y_train, x_val, y_val, x_test, y_test, scaler, train_idx, val_idx, test_idx, config):
    train_results = get_results(model, x_train, y_train, scaler, train_idx)
    val_results = get_results(model, x_val, y_val, scaler, val_idx)
    test_results = get_results(model, x_test, y_test, scaler, test_idx)

    metrics = []
    for split_name, results in zip(['Train', 'Validation', 'Test'], [train_results, val_results, test_results]):
        metrics.extend(compute_metrics(results, split_name))

    metrics_df = pd.DataFrame(metrics, columns=['Split', 'RMSE', 'MAE', 'MAPE (%)'])
    print(metrics_df.to_string(index=False))
    save_metrics(metrics_df, "evaluation_metrics.csv")

    train_rmse = metrics_df.loc[metrics_df['Split'] == 'Train (inversed)', 'RMSE'].values[0]
    val_rmse = metrics_df.loc[metrics_df['Split'] == 'Validation (inversed)', 'RMSE'].values[0]
    test_rmse = metrics_df.loc[metrics_df['Split'] == 'Test (inversed)', 'RMSE'].values[0]
    ratio = val_rmse / train_rmse if train_rmse > 0 else np.inf

    if ratio > 1.5 and val_rmse > 1.5 * test_rmse:
        raise Exception("Model is likely OVERFITTING: Validation error is much higher than training error.")
    elif train_rmse > 2 * test_rmse and val_rmse > 2 * test_rmse:
        raise Exception("Model is likely UNDERFITTING: Both training and validation errors are high.")
    else:
        logging.info("Model fit is acceptable. Proceeding to plot results...")

    plot_all_results(train_results, val_results, test_results, config)

# --- Main Workflow ---
def main(client=None):
    # 1. Data Loading and Feature Engineering
    df, data_file = load_or_download_data(client=client)
    print("Data loaded and indicators added. Proceeding to feature selection...")

    features = df[['close', 'volume', 'RSI_14', 'ROC_1', 'EMA_12']].dropna()
    print("Features selected. Proceeding to scaling and splitting...")

    # 2. Scaling and Splitting
    minmax_cols = ['close', 'volume', 'RSI_14', 'EMA_12']
    std_cols = ['ROC_1']
    scaler = FeatureScaler(minmax_cols, std_cols)
    train_data, val_data, test_data, train_end, val_end, n_cut = split_and_scale(features, scaler, CONFIG)
    print("Data scaled and split. Proceeding to LSTM/GRU input preparation...")

    # 3. Prepare LSTM/GRU Inputs
    x_train, y_train = df_to_x_y_multivariate(train_data, CONFIG['WINDOW_SIZE'])
    x_val, y_val = df_to_x_y_multivariate(val_data, CONFIG['WINDOW_SIZE'])
    x_test, y_test = df_to_x_y_multivariate(test_data, CONFIG['WINDOW_SIZE'])
    print("First 5 rows of train_data:")
    print(pd.DataFrame(train_data, columns=features.columns).head())
    print("\nFirst 5 rows of val_data:")
    print(pd.DataFrame(val_data, columns=features.columns).head())
    print("\nFirst 5 rows of test_data:")
    print(pd.DataFrame(test_data, columns=features.columns).head())
    print("LSTM/GRU inputs prepared. Proceeding to model definition...")
    print(x_train.shape, y_train.shape, x_val.shape, y_val.shape, x_test.shape, y_test.shape)

    # 4. Model Definition
    input_shape = (CONFIG['WINDOW_SIZE'], train_data.shape[1])
    symbol = data_file.split('_')[0] if '_' in data_file else 'model'
    model_filename = f"model_({data_file if isinstance(data_file, str) else data_file}).keras"
    model = build_gru_model(input_shape, CONFIG['LEARNING_RATE'])
    model.summary()
    print("Model defined. Waiting for user confirmation to start training...")

    # 5. Training
    if yes_no_widget("Continue with model generation?") != "y":
        raise RuntimeError("Model generation aborted by user.")

    callbacks = [
        ModelCheckpoint(model_filename, save_best_only=True),
        EarlyStopping(monitor='val_loss', patience=CONFIG['PATIENCE'], restore_best_weights=True)
    ]

    logging.info("Starting model training...")
    history = model.fit(
        x_train, y_train,
        validation_data=(x_val, y_val),
        epochs=CONFIG['EPOCHS'],
        batch_size=CONFIG['BATCH_SIZE'],
        callbacks=callbacks,
        verbose=2,
        shuffle=False
    )
    plot_training_loss(history)
    print("Model training complete. Waiting for user confirmation to start evaluation...")

    # 6. Evaluation
    if yes_no_widget("Continue with model evaluation and plotting?") != "y":
        raise RuntimeError("Model evaluation aborted by user.")

    model_files = [f for f in os.listdir('.') if f.endswith('.h5') or f.endswith('.keras')]
    if not model_files:
        raise FileNotFoundError("No saved model (.h5 or .keras) found in the directory.")
    latest_model_file = max(model_files, key=lambda f: os.path.getmtime(os.path.join('.', f)))
    logging.info(f"Loading latest model: {latest_model_file}")
    model = tf.keras.models.load_model(latest_model_file)

    train_idx = features.iloc[CONFIG['WINDOW_SIZE']:train_end].index
    val_idx = features.iloc[train_end+CONFIG['WINDOW_SIZE']:val_end].index
    test_idx = features.iloc[val_end+CONFIG['WINDOW_SIZE']:n_cut].index

    evaluate_and_plot(
        model, x_train, y_train, x_val, y_val, x_test, y_test,
        scaler, train_idx, val_idx, test_idx, CONFIG
    )
    print("All steps completed successfully. Model evaluation and plotting finished.")

if __name__ == "__main__":
    main(client=client)

In [ ]:
# Forecasting
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# Constants
MINMAX_COLS = ['close', 'volume', 'RSI_14', 'EMA_12']
STD_COLS = ['ROC_1']

class FeatureScaler:
    """Handles feature scaling with different methods for different columns"""
    def __init__(self, minmax_cols, std_cols):
        self.minmax_cols = minmax_cols
        self.std_cols = std_cols
        self.mm_scaler = MinMaxScaler()
        self.std_scaler = StandardScaler()

    def fit(self, df):
        self.mm_scaler.fit(df[self.minmax_cols])
        self.std_scaler.fit(df[self.std_cols])
        return self

    def transform(self, df):
        scaled = df.copy()
        scaled[self.minmax_cols] = self.mm_scaler.transform(df[self.minmax_cols])
        scaled[self.std_cols] = self.std_scaler.transform(df[self.std_cols])
        return scaled

    def inverse_transform_close(self, arr):
        dummy = np.zeros((len(arr), len(self.minmax_cols)))
        dummy[:, 0] = arr
        return self.mm_scaler.inverse_transform(dummy)[:, 0]


def df_to_sequences(df, window_size):
    """Convert DataFrame to sequences for LSTM/GRU training and forecasting"""
    values = df.values
    X, y, current_prices = [], [], []
    for i in range(len(values) - window_size):
        X.append(values[i : i + window_size])
        y.append(values[i + window_size][0])
        current_prices.append(values[i + window_size - 1][0])
    return np.array(X), np.array(y), np.array(current_prices)


def prepare_quarter_data(df, window_size, scaler=None):
    """Prepare data for a quarter with optional pre-fitted scaler"""
    features = df[['close', 'volume', 'ROC_1', 'RSI_14', 'EMA_12']].dropna()
    if scaler is None:
        scaler = FeatureScaler(minmax_cols=MINMAX_COLS, std_cols=STD_COLS).fit(features)
    scaled = scaler.transform(features)
    X, y, current = df_to_sequences(scaled, window_size)
    return X, y, current, scaler


def forecast_quarterly(
    ASSETS, DATA_DIR, MODEL_DIR, OUTPUT_DIR,
    window_size=12,
    date_start='2023-12-01',
    date_end='2025-01-01',
    train_epochs_q0=5,
    train_epochs=5,
    learning_rate=1e-3
):
    # 1. Load raw data
    raw_data = {
        asset: pd.read_csv(
            os.path.join(DATA_DIR, f"{asset}usdt_hourly_indicators.csv"),
            parse_dates=['open_time']
        )
        for asset in ASSETS
    }

    # Define Q0: initial December 2023 period
    q0_start = pd.Timestamp('2023-12-01')
    q0_end = pd.Timestamp('2023-12-31')
    # Define quarters Q1-Q4 of 2024
    quarters = pd.period_range('2024-01-01', '2024-12-31', freq='Q')

    for asset, df in raw_data.items():
        # 2. Filter overall date range
        df = df.set_index('open_time')
        df = df.loc[date_start:date_end].reset_index()

        # Load pre-trained model
        model_path = os.path.join(MODEL_DIR, f"model_{asset}usdt.keras")
        model = tf.keras.models.load_model(model_path)

        all_results = []
        prev_scaler = None

        # === Handle Q0 (Dec 2023) ===
        df_q0 = df[(df['open_time'] >= q0_start) & (df['open_time'] <= q0_end)].copy()
        if not df_q0.empty:
            X0, y0, current0, prev_scaler = prepare_quarter_data(df_q0, window_size)
            # Forecast Q0
            preds0_scaled = model.predict(X0, verbose=0).flatten()
            preds0 = prev_scaler.inverse_transform_close(preds0_scaled)
            actual0 = prev_scaler.inverse_transform_close(y0)
            current_prices0 = prev_scaler.inverse_transform_close(current0)
            timestamps0 = df_q0['open_time'].iloc[window_size:].to_list()
            res0 = pd.DataFrame({
                'date': timestamps0,
                'actual_price': actual0,
                'forecasted_price': preds0,
                'current_price': current_prices0
            })
            all_results.append(res0)
            
            # Train on Q0 before forecasting Q1
            model.compile(
                loss=tf.keras.losses.MeanSquaredError(),
                optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                metrics=[tf.keras.metrics.RootMeanSquaredError()]
            )
            model.fit(
                X0, y0,
                epochs=train_epochs_q0,
                batch_size=32,
                verbose=1
            )

        # 3. Loop through Q1–Q4 of 2024
        for q in quarters:
            q_start = q.start_time
            q_end = q.end_time
            q_df = df[(df['open_time'] >= q_start) & (df['open_time'] <= q_end)].copy()
            if q_df.empty:
                continue

            # Prepare and scale data
            X, y_true, current_scaled, prev_scaler = prepare_quarter_data(
                q_df, window_size, scaler=prev_scaler
            )
            # Forecast
            preds_scaled = model.predict(X, verbose=0).flatten()
            preds = prev_scaler.inverse_transform_close(preds_scaled)
            actuals = prev_scaler.inverse_transform_close(y_true)
            currents = prev_scaler.inverse_transform_close(current_scaled)
            timestamps = q_df['open_time'].iloc[window_size:].to_list()

            res_df = pd.DataFrame({
                'date': timestamps,
                'actual_price': actuals,
                'forecasted_price': preds,
                'current_price': currents
            })
            all_results.append(res_df)

            # Train on this quarter before next
            model.compile(
                loss=tf.keras.losses.MeanSquaredError(),
                optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                metrics=[tf.keras.metrics.RootMeanSquaredError()]
            )
            model.fit(
                X, y_true,
                epochs=train_epochs,
                batch_size=32,
                verbose=1
            )

        # 4. Save all results (Q0 + Q1-Q4) to a single file
        if all_results:
            out = pd.concat(all_results, ignore_index=True)
            os.makedirs(OUTPUT_DIR, exist_ok=True)
            out.to_csv(
                os.path.join(OUTPUT_DIR, f"{asset}usdt_forecasted.csv"),
                index=False
            )

ASSETS = ['btc', 'eth', 'sol', 'avax', 'bnb', 'pca']      # whatever tickers you care about
forecast_quarterly(
    ASSETS,
    DATA_DIR='data',
    MODEL_DIR='models',
    OUTPUT_DIR='Forecast'
)

In [ ]:
# Optimization (best weight allocation plus Sharpe ratio and DAO weight correlation per hour)

import torch
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import time
import os

print("CUDA available:", torch.cuda.is_available())
print("Using device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")
print("🚀 April optimization script started...")

# Settings
ASSETS = ['btc', 'eth', 'sol', 'avax', 'bnb']
PCA_ASSET = 'pca'
RISK_FREE_RATE = 0.044 / (365 * 24)
TOTAL_SAMPLES = 500_000
ALPHAS = [0.5, 1.0, 2.0]
WINDOW_HOURS = 168
CHECKPOINT_PATH = "april_optimization_checkpoint.pkl"
PCA_ANALYSIS_POINTS = 100
TEST_MODE = False

# Date filter for April 2024
def in_april(dt):
    return datetime(2023, 12, 16) <= dt < datetime(2024, 12, 31)

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Dirichlet sampling
def sample_dirichlet_weights(num_samples, num_assets, alpha=1.0, pca_idx=None, pca_weight=None):
    if pca_idx is not None and pca_weight is not None:
        non_pca = np.random.dirichlet([alpha] * (num_assets-1), num_samples)
        W = np.zeros((num_samples, num_assets))
        W[:, pca_idx] = pca_weight
        W[:, np.arange(num_assets) != pca_idx] = (1 - pca_weight) * non_pca
        return W
    return np.random.dirichlet([alpha] * num_assets, num_samples)

# Optimization function
def optimize_weights_torch(mean_vec, cov_matrix, asset_names, rfr, device,
                            num_samples, alphas, pca_analysis=False, pca_idx=None):
    best = {'Sharpe': -np.inf}
    n = len(asset_names)
    if pca_analysis:
        res = []
        pca_ws = np.linspace(0, 1, PCA_ANALYSIS_POINTS)
        for w in pca_ws:
            W = sample_dirichlet_weights(num_samples // PCA_ANALYSIS_POINTS, n, 1.0, pca_idx, w)
            T = torch.tensor(W, dtype=torch.float32, device=device)
            ret = T @ mean_vec
            W1 = T.unsqueeze(1)
            C = cov_matrix.expand(T.size(0), n, n)
            var = torch.bmm(W1, torch.bmm(C, W1.transpose(1,2))).squeeze()
            std = torch.sqrt(var)
            sharpe = (ret - rfr) / (std + 1e-8)
            idx_max = torch.argmax(sharpe)
            res.append({
                'PCA Weight': w,
                'Sharpe': sharpe[idx_max].item()
            })
        return None, pd.DataFrame(res)
    else:
        for a in alphas:
            W = sample_dirichlet_weights(num_samples // len(alphas), n, a)
            T = torch.tensor(W, dtype=torch.float32, device=device)
            ret = T @ mean_vec
            W1 = T.unsqueeze(1)
            C = cov_matrix.expand(T.size(0), n, n)
            var = torch.bmm(W1, torch.bmm(C, W1.transpose(1,2))).squeeze()
            std = torch.sqrt(var)
            sharpe = (ret - rfr) / (std + 1e-8)
            sharpe[torch.isnan(sharpe)] = -float('inf')
            sharpe[torch.isinf(sharpe)] = -float('inf')
            im = torch.argmax(sharpe)
            if sharpe[im] > best['Sharpe']:
                best = {**{f'{asset} Weight': T[im][i].item() for i, asset in enumerate(asset_names)},
                        'Sharpe': sharpe[im].item()}
        return best, None

# Load data
returns_f, returns_a, prices = {}, {}, {}
for asset in ASSETS + [PCA_ASSET]:
    path = f"Forecast/{asset}usdt_forecasted.csv"
    if not os.path.exists(path):
        print(f"⚠️ Missing {asset}, skipping")
        continue
    df = pd.read_csv(path, parse_dates=['date']).dropna()
    df['forecasted_return'] = (df['forecasted_price'] - df['current_price']) / df['current_price']
    df['actual_return']   = (df['actual_price']   - df['current_price']) / df['current_price']
    returns_f[asset] = df.set_index('date')['forecasted_return']
    returns_a[asset] = df.set_index('date')['actual_return']
    prices[asset]     = df.set_index('date')['actual_price']

# Align dates and filter April
f_df = pd.DataFrame(returns_f).dropna()
a_df = pd.DataFrame(returns_a).dropna()
e_df = pd.DataFrame(prices).pct_change().shift(1).dropna()
dates = sorted(set(f_df.index) & set(a_df.index) & set(e_df.index))
apr_dates = [d for d in dates if in_april(d)]

# Prepare results
corr_records = []
best_forecasted = {'Sharpe': -np.inf}
best_expected   = {'Sharpe': -np.inf}
n = len(ASSETS) + 1
pca_i = list(f_df.columns).index(PCA_ASSET)

# Main loop over April hours
for i, date in enumerate(apr_dates):
    print(f"🕒 [{i+1}/{len(apr_dates)}] Optimization started at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')} for hour: {date}")
    start = date - timedelta(hours=WINDOW_HOURS)
    f_win = f_df.loc[start:date]
    a_win = a_df.loc[start:date]
    e_win = e_df.loc[start:date]
    if len(f_win) < WINDOW_HOURS or len(e_win) < WINDOW_HOURS:
        continue
    f_mu = torch.tensor(f_win.mean().values, dtype=torch.float32, device=device)
    f_cov= torch.tensor(f_win.cov().values, dtype=torch.float32, device=device).unsqueeze(0)
    a_mu = torch.tensor(e_win.mean().values, dtype=torch.float32, device=device)
    a_cov= torch.tensor(a_win.cov().values, dtype=torch.float32, device=device).unsqueeze(0)
    # Correlation analysis for this hour
    _, f_pca = optimize_weights_torch(f_mu, f_cov, list(f_win.columns), RISK_FREE_RATE, device, TOTAL_SAMPLES, ALPHAS, True, pca_i)
    _, a_pca = optimize_weights_torch(a_mu, a_cov, list(a_win.columns), RISK_FREE_RATE, device, TOTAL_SAMPLES, ALPHAS, True, pca_i)
    corr_records.append({
        'date': date,
        'forecasted_corr': f_pca['PCA Weight'].corr(f_pca['Sharpe']),
        'expected_corr':   a_pca['PCA Weight'].corr(a_pca['Sharpe'])
    })
    print(f"✅ Correlation analysis done for {date} — Forecasted corr: {corr_records[-1]['forecasted_corr']:.4f}, Expected corr: {corr_records[-1]['expected_corr']:.4f}")
    # Track best overall
    f_best, _ = optimize_weights_torch(f_mu, f_cov, list(f_win.columns), RISK_FREE_RATE, device, TOTAL_SAMPLES, ALPHAS)
    a_best, _ = optimize_weights_torch(a_mu, a_cov, list(a_win.columns), RISK_FREE_RATE, device, TOTAL_SAMPLES, ALPHAS)
    if f_best['Sharpe'] > best_forecasted['Sharpe']: best_forecasted = {**f_best, 'date': date}
    if a_best['Sharpe'] > best_expected['Sharpe']:   best_expected   = {**a_best, 'date': date}

# Save correlation CSV
pd.DataFrame(corr_records).to_csv('expected_weightVSsharpe_correlation.csv', index=False)
print("📂 Saved per-hour correlations.")

# Save best allocations
best_f_df = pd.DataFrame([best_forecasted])
best_a_df = pd.DataFrame([best_expected])
best_f_df.to_csv('best_year_forecasted_portfolios.csv', index=False)
best_a_df.to_csv('best_year_expected_portfolios.csv', index=False)
print("📂 Saved best April portfolios.")


In [ ]:
# DAO weight Allocation and Sharpe Ratio progression plotting
# Event divided plot
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime
import os

# =============================================
# 1. Data Loading and Preparation
# =============================================

# Load portfolio weights and Sharpe data
df_weights = pd.read_csv('best_pca_forecasted_portfolios.csv', parse_dates=['date'])
layer1_assets = ['btc', 'eth', 'sol', 'avax', 'bnb']
dao_assets   = ['pca']

# Load price data for assets
assets = layer1_assets + dao_assets
price_data = {}
for asset in assets:
    file_path = os.path.join('Forecast', f'{asset}usdt_forecasted.csv')
    try:
        df_price = pd.read_csv(file_path, parse_dates=['date'])
        price_data[asset] = df_price.set_index('date')['actual_price']
    except (FileNotFoundError, KeyError):
        # fallback to 'close'
        try:
            price_data[asset] = df_price.set_index('date')['close']
        except:
            print(f"Skipping {asset}: no price column found.")

prices  = pd.concat(price_data, axis=1)
returns = prices.pct_change().dropna()

# =============================================
# 2. Weights & Returns Alignment
# =============================================
df_weights['Layer1_Weight'] = df_weights[[f'{a} Weight' for a in layer1_assets]].sum(axis=1)
df_weights['DAO_Weight']    = df_weights[[f'{a} Weight' for a in dao_assets]].sum(axis=1)

# Daily mean weights
df_daily = (df_weights
    .assign(day=lambda d: d['date'].dt.date)
    .groupby('day')[['Layer1_Weight','DAO_Weight','Sharpe']]
    .mean()
    .rename_axis('day')
    .reset_index()
    .assign(date=lambda d: pd.to_datetime(d['day']))
)

df_combined = df_daily.merge(returns, on='date', how='left')
df_combined['Layer1_Weight_Smoothed'] = df_combined['Layer1_Weight'].rolling(3, min_periods=1).mean()
df_combined['DAO_Weight_Smoothed']    = df_combined['DAO_Weight'].rolling(3, min_periods=1).mean()


# =============================================
# 3. Combined Visualization with Event Shading
# =============================================
fig, (ax1, ax2) = plt.subplots(2,1, figsize=(20,18), sharex=True)
plt.suptitle('DAO Asset Analysis Dashboard', y=1.02, fontsize=18)

# -- Top: DAO Weight --
ax1.plot(df_combined['date'], df_combined['DAO_Weight_Smoothed'],
         label='DAO Weight', linewidth=2, color='#ff7f0e')
ax1.set_ylabel('DAO Weight')
ax1.set_title('DAO Weight Over Time')

# -- Mid: Sharpe Ratio --
ax2.plot(df_combined['date'], df_combined['Sharpe'],
         label='Sharpe Ratio', linewidth=2, color='purple')
ax2.set_ylabel('Sharpe Ratio')
ax2.set_title('Portfolio Sharpe Ratio')

# -- X‑axis formatting --
for ax in (ax1, ax2):
    ax.grid(True)
    ax.legend(loc='upper right')
    ax.xaxis.set_major_locator(mdates.MonthLocator())
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))
    ax.xaxis.set_minor_locator(mdates.WeekdayLocator(byweekday=mdates.MO))

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
# DAO weight allocation and Sharpe Ratio correlation
import pandas as pd
import matplotlib.pyplot as plt

def plot_normal_scatter(csv_file_path: str = 'pca_expected_effect.csv'):
    """
    Creates a normal scatter plot with:
    - X-axis: Sharpe Ratio
    - Y-axis: PCA Weight
    (Raw data points only, no enforced centering)
    """
    
    # Read and prepare data
    try:
        data = pd.read_csv(csv_file_path)
        sharpe = data['Sharpe'].values
        pca_weight = data['PCA Weight'].values
    except Exception as e:
        print(f"Error reading file: {e}")
        return
    
    # Create plot with professional styling
    plt.figure(figsize=(10, 6))
    
    # Plot raw data points with improved styling
    plt.scatter(sharpe, pca_weight, 
                color='steelblue', 
                s=80,
                alpha=0.8,
                edgecolors='white',
                linewidth=1,
                label='Portfolio Allocations')
    
    # Add light zero lines (optional)
    plt.axhline(0, color='gray', linestyle='--', linewidth=0.5, alpha=0.5)
    plt.axvline(0, color='gray', linestyle='--', linewidth=0.5, alpha=0.5)
    
    # Automatic axis limits based on data
    x_padding = (sharpe.max() - sharpe.min()) * 0.1
    y_padding = (pca_weight.max() - pca_weight.min()) * 0.1
    
    plt.xlim(sharpe.min() - x_padding, sharpe.max() + x_padding)
    plt.ylim(pca_weight.min() - y_padding, pca_weight.max() + y_padding)
    
    # Labels and title
    plt.xlabel('Sharpe Ratio', fontsize=12)
    plt.ylabel('PCA Weight', fontsize=12)
    plt.title('PCA Weight vs Sharpe Ratio Distribution (Expected Optimization)', fontsize=14, pad=15)
    
    # Grid and aesthetics
    plt.grid(True, linestyle=':', alpha=0.3)
    
    # Remove top/right spines and adjust left/bottom
    for spine in ['top', 'right']:
        plt.gca().spines[spine].set_visible(False)
    for spine in ['left', 'bottom']:
        plt.gca().spines[spine].set_linewidth(0.5)
        plt.gca().spines[spine].set_color('gray')
    
    # Add data density indicators if many points
    if len(sharpe) > 50:
        from scipy.stats import gaussian_kde
        try:
            xy = np.vstack([sharpe, pca_weight])
            z = gaussian_kde(xy)(xy)
            plt.scatter(sharpe, pca_weight, c=z, s=40, cmap='viridis', alpha=0.6)
            plt.colorbar(label='Data Density')
        except:
            pass
    
    plt.tight_layout()
    plt.show()

# Run the function
plot_normal_scatter()

In [ ]:
# DAO weight allocation and Standard Deviation correlation
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde

def plot_std_scatter(csv_file_path: str = 'pca_forecasted_effect.csv'):
    """
    Creates a scatter plot with:
    - X-axis: PCA Weight
    - Y-axis: Standard Deviation
    - Color-coded data density
    """
    
    # Read and prepare data
    try:
        data = pd.read_csv(csv_file_path)
        std_dev = data['Standard Deviation'].values  # Standard Deviation values
        pca_weight = data['PCA Weight'].values
    except Exception as e:
        print(f"Error reading file: {e}")
        return
    
    # Create plot with professional styling
    plt.figure(figsize=(10, 6))
    
    # Calculate point density for coloring (using swapped axes)
    xy = np.vstack([pca_weight, std_dev])  # Swapped order
    z = gaussian_kde(xy)(xy)
    
    # Sort points by density (so densest points appear on top)
    idx = z.argsort()
    pca_weight, std_dev, z = pca_weight[idx], std_dev[idx], z[idx]
    
    # Plot data points with density-based coloring (axes swapped)
    scatter = plt.scatter(pca_weight, std_dev, 
                          c=z, 
                          s=80,
                          alpha=0.7,
                          cmap='viridis',
                          edgecolors='w',
                          linewidth=0.8)
    
    # Add colorbar
    cbar = plt.colorbar(scatter)
    cbar.set_label('Data Density', fontsize=12)
    
    # Add zero lines
    plt.axhline(0, color='gray', linestyle='--', linewidth=0.8, alpha=0.7)
    plt.axvline(0, color='gray', linestyle='--', linewidth=0.8, alpha=0.7)
    
    # Set axis limits (using swapped axes)
    x_padding = (pca_weight.max() - pca_weight.min()) * 0.1
    y_padding = (std_dev.max() - std_dev.min()) * 0.1
    plt.xlim(pca_weight.min() - x_padding, pca_weight.max() + x_padding)
    plt.ylim(std_dev.min() - y_padding, std_dev.max() + y_padding)
    
    # Labels and title (updated for swapped axes)
    plt.xlabel('PCA Weight', fontsize=12)
    plt.ylabel('Standard Deviation', fontsize=12)
    plt.title('Portfolio Standard Deviation vs PCA Weight', fontsize=14, pad=15)
    
    # Grid and aesthetics
    plt.grid(True, linestyle=':', alpha=0.3)
    
    # Remove top/right spines and adjust left/bottom
    for spine in ['top', 'right']:
        plt.gca().spines[spine].set_visible(False)
    for spine in ['left', 'bottom']:
        plt.gca().spines[spine].set_linewidth(0.5)
        plt.gca().spines[spine].set_color('gray')
    
    # Adjust colorbar ticks
    cbar.ax.tick_params(labelsize=10)
    
    plt.tight_layout()
    plt.show()

# Run the function
plot_std_scatter()

In [ ]:
# Market regime base - Three Theory Analysis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

# === 1. Load data ===
opt_df = pd.read_csv("expected_weightVSsharpe_correlation.csv",
                     parse_dates=["date"], index_col="date")
dao = pd.read_csv("data/pcausdt_hourly_indicators.csv",
                  parse_dates=["open_time"], index_col="open_time")
l1 = pd.read_csv("data/lyrusdt_hourly_indicators.csv",
                 parse_dates=["open_time"], index_col="open_time")

common = opt_df.index.intersection(dao.index).intersection(l1.index)
opt_df, dao, l1 = opt_df.loc[common], dao.loc[common], l1.loc[common]

# === 2. Compute hourly indicators ===
dao_ret = dao["close"].pct_change()
l1_ret = l1["close"].pct_change()
drawdown = (l1["close"] - l1["close"].cummax()) / l1["close"].cummax()
mark_corr = dao_ret.rolling(24).corr(l1_ret)

betas, idx = [], []
for t in range(24, len(dao_ret)):
    wdr = dao_ret.iloc[t-24:t]
    wlr = l1_ret.iloc[t-24:t]
    if wdr.isnull().any() or wlr.isnull().any():
        betas.append(np.nan)
    else:
        X = sm.add_constant(wlr)
        betas.append(sm.OLS(wdr, X).fit().params[1])
    idx.append(dao_ret.index[t])
beta_series = pd.Series(betas, index=idx)

illiquidity = (dao_ret.abs() / dao["volume"]).replace([np.inf, -np.inf], np.nan)

df_hr = pd.DataFrame({
    "opt_corr":    opt_df["expected_corr"],
    "drawdown":    drawdown,
    "mark_corr":   mark_corr,
    "beta":        beta_series,
    "illiquidity": illiquidity
}).dropna()

for col in ["drawdown", "mark_corr", "beta", "illiquidity"]:
    df_hr[f"{col}_lag"] = df_hr[col].shift(1)
df_hr = df_hr.dropna()

# === 3. Determine regime via moving average crossover ===
price = l1["close"].resample("D").last()
fast = price.rolling(window=50, min_periods=1).mean()
slow = price.rolling(window=200, min_periods=1).mean()

daily_regimes = pd.Series(index=price.index, dtype="object")
daily_regimes[fast > slow] = "bull"
daily_regimes[fast < slow] = "bear"
daily_regimes[(fast == slow)] = "neutral"

monthly_ma_regimes = (daily_regimes
                      .to_frame("regime")
                      .assign(month=lambda df: df.index.to_period("M"))
                      .groupby("month")["regime"]
                      .agg(lambda x: x.mode()[0]))

# === 4. Correlation by month ===
df_hr["month"] = df_hr.index.to_period("M")
df_hr["regime"] = df_hr["month"].map(monthly_ma_regimes)

records = []

for mon, grp in df_hr.groupby("month"):
    regime = grp["regime"].iloc[0]
    corr_draw  = grp[["opt_corr","drawdown_lag"]].corr().iloc[0,1]
    corr_mark  = grp[["opt_corr","mark_corr_lag"]].corr().iloc[0,1]
    corr_beta  = grp[["opt_corr","beta_lag"]].corr().iloc[0,1]
    corr_illiq = grp[["opt_corr","illiquidity_lag"]].corr().iloc[0,1]
    
    records.append({
        "month": mon.strftime("%Y-%m"),
        "regime": regime,
        "draw": corr_draw,
        "mark": corr_mark,
        "beta": corr_beta,
        "ill":  corr_illiq
    })

# === 5. Summarize correlation by regime ===
df_results = pd.DataFrame(records)
summary = df_results.groupby("regime")[["draw", "mark", "beta", "ill"]].mean().round(3)

print("=== Average Lagged Correlations by Regime ===")
print(summary)
